# Making your own `EventData`

In this example we will download data from [GWOSC](https://www.gw-openscience.org/) and make our own `EventData` (instead of loading one of the precomputed `npz` files).

In [1]:
%matplotlib notebook

In [2]:
path_to_cogwheel = '..'

import sys
sys.path.append(path_to_cogwheel)

import subprocess
from matplotlib import pyplot as plt

import gwosc

from cogwheel import data
from cogwheel.posterior import Posterior

In [3]:
eventname = 'GW170817'

To download the default strain data from GWOSC, we could use the following lines and skip the next cell:

    data.download_timeseries(eventname)
    filenames = sorted((data.GWOSC_FILES_DIR/eventname).glob('*.hdf5'))
   
Here we will download [specific hdf5 data files from GWOSC](https://www.gw-openscience.org/eventapi/html/O1_O2-Preliminary/GW170817/v2) instead (with the glitch at Livingston cleaned):

In [4]:
urls = gwosc.locate.get_event_urls('GW170817', version=2)  # Cleaned GW170817

outdir = data.GWOSC_FILES_DIR/eventname
subprocess.run(['wget', '-P', outdir, *urls])

filenames = [outdir/url.split('/')[-1] for url in urls]

--2022-10-21 14:18:17--  https://www.gw-openscience.org/eventapi/json/O1_O2-Preliminary/GW170817/v2/H-H1_LOSC_CLN_4_V1-1187007040-2048.hdf5
Resolving www.gw-openscience.org (www.gw-openscience.org)... 131.215.113.73
Connecting to www.gw-openscience.org (www.gw-openscience.org)|131.215.113.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57932612 (55M) [application/octet-stream]
Saving to: ‘/data/tislam/works/KITP/final_repo/cogwheel/tutorials/../cogwheel/data/gwosc_files/GW170817/H-H1_LOSC_CLN_4_V1-1187007040-2048.hdf5.26’

     0K .......... .......... .......... .......... ..........  0%  346K 2m43s
    50K .......... .......... .......... .......... ..........  0%  676K 2m3s
   100K .......... .......... .......... .......... ..........  0% 88.2M 82s
   150K .......... .......... .......... .......... ..........  0%  688K 82s
   200K .......... .......... .......... .......... ..........  0% 68.6M 66s
   250K .......... .......... .......... .......... ..

In [6]:
filenames

[PosixPath('/data/tislam/works/KITP/final_repo/cogwheel/tutorials/../cogwheel/data/gwosc_files/GW170817/H-H1_LOSC_CLN_4_V1-1187007040-2048.hdf5'),
 PosixPath('/data/tislam/works/KITP/final_repo/cogwheel/tutorials/../cogwheel/data/gwosc_files/GW170817/L-L1_LOSC_CLN_4_V1-1187007040-2048.hdf5'),
 PosixPath('/data/tislam/works/KITP/final_repo/cogwheel/tutorials/../cogwheel/data/gwosc_files/GW170817/V-V1_LOSC_CLN_4_V1-1187007040-2048.hdf5')]

In [7]:
import h5py

In [10]:
f = h5py.File('/data/tislam/works/KITP/final_repo/cogwheel/cogwheel/data/gwosc_files/GW170817/H-H1_LOSC_CLN_4_V1-1187007040-2048.hdf5', 'r')

In [12]:
f.keys()

<KeysViewHDF5 ['meta', 'quality', 'strain']>

In [23]:
def printname(name):
    print(name)

f.visit(printname)

meta
meta/Description
meta/DescriptionURL
meta/Detector
meta/Duration
meta/GPSstart
meta/Observatory
meta/Type
meta/UTCstart
quality
quality/detail
quality/injections
quality/injections/InjDescriptions
quality/injections/InjShortnames
quality/injections/Injmask
quality/simple
quality/simple/DQDescriptions
quality/simple/DQShortnames
quality/simple/DQmask
strain
strain/Strain


In [28]:
f['quality/injections/InjDescriptions'][()]

array([b'Passes NO_CBC_HW_INJ test',
       b'Passes NO_BURST_HW_INJ test',
       b'Passes NO_DETCHAR_HW_INJ test',
       b'Passes NO_CW_HW_INJ test',
       b'Passes NO_STOCH_HW_INJ test'], dtype='|S29')

In [30]:
f['meta/Duration'][()]

2048

In [31]:
f['meta/GPSstart'][()]

1187007040

In [33]:
f['meta/Detector'][()]

b'H1'

In [ ]:
detector_names = ''.join(filename.name[0] for filename in filenames)
tgps = gwosc.datasets.event_gps(eventname)

Make `EventData` making sure we keep a length of data enough to contain the signal, and that the Nyquist frequency is high enough to capture all available SNR.

This estimates the noise power spectrum with the Welch method from the full file and crops, high-passes and whitens a chunk of data. The duration of the chunk of data, that of the Welch segments and the Nyquist frequency are determined by the arguments to `from_timeseries`.

In [ ]:
event_data = data.EventData.from_timeseries(
    filenames, eventname.split('-')[0], detector_names, tgps, t_before=128., fmax=1600.)

Plot whitening filter (inverse of the noise amplitude spectral density times a high-pass)

In [ ]:
plt.figure()
plt.plot(event_data.frequencies, event_data.wht_filter.T,
         label=list(event_data.detector_names))
plt.legend()

plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Whitening filter ($\sqrt{\rm Hz}$)')
plt.xlim(0)
plt.ylim(0);

Plot spectrogram (full file and zoom-in)

In [ ]:
event_data.specgram()

In [ ]:
event_data.specgram((-1.5, .5), nfft=200)

We can use this `EventData` to make a `Posterior` object.

*Note:* `mchirp_guess` is detector frame chirp-mass, should be less than a few sigmas away from the truth. This becomes important for low mass systems like GW170817 whose chirp mass is very well measured.

In [ ]:
mchirp_guess = 1.198
post = Posterior.from_event(event_data, mchirp_guess, 'IMRPhenomXPHM', 'LVCPrior')

We can save the event data and/or the posterior to use later:
    
    event_data.to_npz()
    post.to_json(dirname)  # Then load with cogwheel.utils.read_json()